In [1]:

import json, requests
import string
import re
import gzip
import shutil
import urllib.request 
import pandas as pd
# import imdbpy


# Downloading IMDB dataset of names
# imdb_url = 'https://datasets.imdbws.com/name.basics.tsv.gz'
# imdb_file = requests.get(imdb_url, stream=True)
# open('../data/namebasics.tsv.gz', 'wb').write(imdb_file.content)    # https://www.tutorialspoint.com/downloading-files-from-web-using-python

# with gzip.open('../data/namebasics.tsv.gz', 'rb') as f_in:          # https://stackoverflow.com/questions/31028815/how-to-unzip-gz-file-using-python
#     with open('../data/namebasics.tsv', 'wb') as f_out:
#         shutil.copyfileobj(f_in, f_out)


# df = pd.read_csv("../data/the_oscar_award.csv")
imdb = pd.read_csv("../data/namebasics.tsv", sep='\t')
oscars = pd.read_csv("../data/oscar_winners_clean.csv")
oscars["primaryName"]=oscars["name"]

del imdb["knownForTitles"]

# Merging data
# winners = pd.merge(oscars, imdb, how='inner', on=['primaryName'])
# ['year_ceremony', 'category', 'name', 'film', 'winner', 'primaryName','nconst', 'birthYear', 'deathYear', 'primaryProfession']
# winners.to_csv('../data/clean_names.csv', index=False)

# Calculating age they won the award
# subtract birth year from ceremony date, if winner==No then make won year 0
# winners["winAge"] = winners['year_ceremony'].notna().astype(int) - winners['birthYear'].notna().astype(int)
# Removing duplicates
# winners = winners.drop(winners[(winners['birthYear'])].index)
# winners = winners.groupby(['year_ceremony', 'category','birthYear'], as_index=False)

# # Current age --> Nominees are invited to the Academy
# # If deathYear empty then make current age Null

# # print(winners.head(if not winners["birthYear"]))
# print(winners.head(20))
# winners.to_csv('../data/clean_names.csv', index=False)

In [21]:
winners = pd.merge(oscars, imdb, how='inner', on=['primaryName'])
winners.head(20)

,year_ceremony,category,name,film,winner,primaryName,nconst,birthYear,deathYear,primaryProfession
0,1928,ACTOR,Richard Barthelmess,The Noose,False,Richard Barthelmess,nm0001932,1895,1963,"actor,soundtrack,producer"
1,1928,ACTOR,Emil Jannings,The Last Command,True,Emil Jannings,nm0417837,1884,1950,"actor,producer,art_department"
2,1928,ACTRESS,Louise Dresser,A Ship Comes In,False,Louise Dresser,nm0237571,1878,1965,actress
3,1928,ACTRESS,Janet Gaynor,7th Heaven,True,Janet Gaynor,nm0310980,1906,1984,"actress,soundtrack"
4,1928,ACTRESS,Janet Gaynor,7th Heaven,True,Janet Gaynor,nm1068510,\N,\N,"editor,editorial_department,production_manager"
5,1928,ACTRESS,Janet Gaynor,7th Heaven,True,Janet Gaynor,nm5045096,\N,\N,visual_effects
6,1938,ACTRESS,Janet Gaynor,A Star Is Born,False,Janet Gaynor,nm0310980,1906,1984,"actress,soundtrack"
7,1938,ACTRESS,Janet Gaynor,A Star Is Born,False,Janet Gaynor,nm1068510,\N,\N,"editor,editorial_department,production_manager"
8,1938,ACTRESS,Janet Gaynor,A Star Is Born,False,Janet Gaynor,nm5045096,\N,\N,visual_effects
9,1928,ACTRESS,Gloria Swanson,Sadie Thompson,False,Gloria Swanson,nm0841797,1899,1983,"actress,soundtrack,producer"


In [108]:
# win = winners
win = winners.drop_duplicates(['name','category', 'film'], keep='first').reset_index() # dropping duplicates with the same name, category, year, and film. 2 people have been nominated for 2 different films
win = win.replace(r'\\N', 0, regex=True)


# Age when they were nominated. NOTE: This might not have been their actual age as we don't have their birth days. A better approach would be to find a dataset with their age at nomination
win["ceremonyAge"]=pd.to_numeric(win['year_ceremony'], errors='coerce')-pd.to_numeric(win['birthYear'], errors='coerce')  # Age at the time of the ceremony. NOTE: Some people were nominated after they died, how should we handle this?
win.loc[ (win.ceremonyAge>1000), 'ceremonyAge'] = 0  ## Some entries don't have a birth so making them 0
# win['birthYear']=pd.to_datetime(win['birthYear'], format='%Y', errors='coerce').year
# win['deathYear']=pd.to_datetime(win['deathYear'], format='%Y', errors='coerce')

# ## Current age --> Nominees are invited to the Academy, this would give us an idea of what the voting population is
# # If deathYear empty then make current age Null

# # Checking if nominees are alive
win.loc[ (win.deathYear==0) & (win.birthYear!=0), 'alive'] = True   
win.loc[ (win.deathYear!=0) & (win.birthYear!=0), 'alive'] = False
win.loc[ (win.deathYear==0) & (win.birthYear==0), 'alive'] = False ## if no birth and death year, then not alive
# win = win.replace(win.nan, 0, regex=True)
win['birthYear'] = win['birthYear'].astype(int)

# # Calculating current age. subtract birthYear from currentYear then if alive=
win['currentYear']=int(2021)

# 
win["currentAge"] = win['currentYear']- win['birthYear']
win.loc[win.alive==False, 'currentAge'] = 0

# # Deleting duplicate name column
del(win["primaryName"])
# win.loc[]

# # print(winners.head(if not winners["birthYear"]))
# # print(win.head(20))
win.head(50)
win.to_csv('../data/clean_names.csv', index=False)
# # l=win['birthYear']
# # l.head(50)
# winners.head(20)

In [85]:
win['currentYear']=float(2021)

win['birthYear'].index=pd.to_datetime(win['birthYear'], format='%Y', errors='ignore')

win.head(50)

,index,year_ceremony,category,name,film,winner,primaryName,nconst,birthYear,deathYear,primaryProfession,currentYear
0,0,1928,ACTOR,Richard Barthelmess,The Noose,False,Richard Barthelmess,nm0001932,1895,1963,"actor,soundtrack,producer",2021.0
1,1,1928,ACTOR,Emil Jannings,The Last Command,True,Emil Jannings,nm0417837,1884,1950,"actor,producer,art_department",2021.0
2,2,1928,ACTRESS,Louise Dresser,A Ship Comes In,False,Louise Dresser,nm0237571,1878,1965,actress,2021.0
3,3,1928,ACTRESS,Janet Gaynor,7th Heaven,True,Janet Gaynor,nm0310980,1906,1984,"actress,soundtrack",2021.0
4,6,1938,ACTRESS,Janet Gaynor,A Star Is Born,False,Janet Gaynor,nm0310980,1906,1984,"actress,soundtrack",2021.0
5,9,1928,ACTRESS,Gloria Swanson,Sadie Thompson,False,Gloria Swanson,nm0841797,1899,1983,"actress,soundtrack,producer",2021.0
6,12,1930,ACTRESS,Gloria Swanson,The Trespasser,False,Gloria Swanson,nm0841797,1899,1983,"actress,soundtrack,producer",2021.0
7,15,1951,ACTRESS,Gloria Swanson,Sunset Blvd.,False,Gloria Swanson,nm0841797,1899,1983,"actress,soundtrack,producer",2021.0
8,18,1928,ART DIRECTION,Rochus Gliese,Sunrise,False,Rochus Gliese,nm0322843,1891,1978,"art_director,director,production_designer",2021.0
9,19,1928,ART DIRECTION,William Cameron Menzies,The Dove;,True,William Cameron Menzies,nm0580017,1896,1957,"art_director,director,art_department",2021.0
